# Geopy Use and Examples

In [15]:
import csv, time, pprint
import geopy
print "Running geopy version: {0}".format(geopy.__version__)


Running geopy version: 1.11.0


### Each geopy geocoder is its own class

In [16]:
goog_geocoder = geopy.GoogleV3(scheme="http")

create instance of GoogleV3 API class, named my_geocoder. 
the new instance has a method called "geocode".
executing the geocode method with an address as a parameter

In [17]:
location = goog_geocoder.geocode("516 High Street, Bellingham, WA")

print location.address



516 High St, Bellingham, WA 98225, USA


In [18]:
print "{0}, {1}".format(location.longitude, location.latitude)

-122.485818, 48.7378138


In [19]:
pprint.pprint( location.raw )

{u'address_components': [{u'long_name': u'516',
                          u'short_name': u'516',
                          u'types': [u'street_number']},
                         {u'long_name': u'High Street',
                          u'short_name': u'High St',
                          u'types': [u'route']},
                         {u'long_name': u'Bellingham',
                          u'short_name': u'Bellingham',
                          u'types': [u'locality', u'political']},
                         {u'long_name': u'Whatcom County',
                          u'short_name': u'Whatcom County',
                          u'types': [u'administrative_area_level_2',
                                     u'political']},
                         {u'long_name': u'Washington',
                          u'short_name': u'WA',
                          u'types': [u'administrative_area_level_1',
                                     u'political']},
                         {u'long_name': u'Uni

In [20]:
location = goog_geocoder.reverse("38.8977, -77.0365", exactly_one = True)

In [21]:
print location.address


1600 Pennsylvania Ave NW, Washington, DC 20500, USA


In [22]:
#Geocode file is simply a list of addresses in CSV format.
my_address_list = "add_list.csv" #path to input csv

output = "output.csv" #path to output file
#empty containers we'll use later for comparing list items
add_list = [] #empty address list
rand_list = []

#creating a list of addresses we want to geocode from a CSV file
with open(my_address_list, 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        add_list.append(row[0])

Open the CSV file and fill a list with the addresses to Geocode

# Sort the list to decrease the calls to the API

In [23]:
print len(add_list)
add_list.sort()

207


In [24]:
#match is used to check if we've already geocoded the address
match = []
add = 0
lat = 0
longitude = 0

In [25]:
with open(output, 'wb') as cs_out:
    writer = csv.writer(cs_out, delimiter=',')
    for i in add_list:
        #if the current address matches the previous one, we use the same
        #geocoding info so we don't spam the API needlessly
        #print "previous address: {0}".format(add)
        #print "current address: {0}".format(i)
        if add == i:
            print "Matched"
            writer.writerow([add, lat, longitude])
            
        else:
            try:
                #first we'll try google's api, but if they reject our request
                #then it will throw an exception, and the "except" statement
                #below will catch it and change our geocoder to the OSM one
                geolocator = geopy.GoogleV3(scheme="http")
                location = geolocator.geocode(i)
                writer.writerow([i, location.latitude, location.longitude])
                print "using Google Geocoder"
                print "{2}: Latitude = {0}, longitude = {1}".format(location.latitude, location.longitude, location.address)
            except Exception as e:
                geolocator = geopy.Nominatim(scheme="http")
                location = geolocator.geocode(i)
                writer.writerow([i, location.latitude, location.longitude])
                print "Using OSM"
                print "{2}: Latitude = {0}, longitude = {1}".format(location.latitude, location.longitude, location.address)
                #We include a finally statement because no matter which geocoder we
                #use we want to sleep for 5 seconds to not spam the API, and we
                #need to write our locations to the disc, in this case a CSV
                print e
            finally: 
                time.sleep(2)
                add = i
                lat = location.latitude
                longitude = location.longitude

using Google Geocoder
1089 WA-4, Naselle, WA 98638, USA: Latitude = 46.363392, longitude = -123.767499
Matched
using Google Geocoder
1092 WA-4, Naselle, WA 98638, USA: Latitude = 46.3625473, longitude = -123.767886
Matched
Matched
using Google Geocoder
1107 Salmon Creek Rd, Naselle, WA 98638, USA: Latitude = 46.407071, longitude = -123.677273
Matched
Matched
using Google Geocoder
1112 WA-4, Naselle, WA 98638, USA: Latitude = 46.360534, longitude = -123.767122
using Google Geocoder
1172 WA-4, Naselle, WA 98638, USA: Latitude = 46.35672, longitude = -123.760261
using Google Geocoder
1172 WA-4, Naselle, WA 98638, USA: Latitude = 46.35672, longitude = -123.760261
Matched
Matched
Matched
using Google Geocoder
1192 WA-4, Naselle, WA 98638, USA: Latitude = 46.354911, longitude = -123.759188
Matched
Matched
Matched
Matched
Matched
using Google Geocoder
1198 WA-4, Naselle, WA 98638, USA: Latitude = 46.355582, longitude = -123.758032
Matched
using Google Geocoder
1829 198th Pl, Long Beach, WA 98

AttributeError: 'NoneType' object has no attribute 'latitude'